In [ ]:
# default_exp aoc
from nbdev import *
from nbdev.showdoc import *

# Advent of Code Utils

> A collection of somewhat handy functions to make your AoC puzzle life solving a bit easier

## Quality of Life

In [ ]:
#export
from collections.abc import Iterable
from collections import namedtuple, deque
import contextlib
import copy
from functools import reduce
import hashlib
import heapq
import logging
from math import sqrt, gcd
from pathlib import Path
import time

import pandas as pd
import numpy as np

In [ ]:
#export 
import pickle

DATA_DIR = Path('data')
def load(filename):
    f = open(DATA_DIR/filename,"rb")
    return pickle.load(f)
    
def save(data, filename):
    with open(DATA_DIR/filename, 'wb') as handle:
        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

NameError: name 'Path' is not defined

function to export files easily

In [ ]:
#export
def to_int(inp): 
    """ 
        returns items converted to int if possible also works for tuples\n
        also works recursively
        watch out because passing a string '12t' will be ripped into a list [1,2,t]
    """
    if isinstance(inp,str):
        print('watch out string will be converted into list of characters')
    if isinstance(inp[0],list):
        return list(to_int(l) for l in inp)
    if isinstance(inp[0],tuple):
        return tuple(to_int(l) for l in inp)

    out = []
    for i in inp:
        try:
            out.append(int(i))
        except ValueError:
            out.append(i)
    if isinstance(inp,tuple): return tuple(out)
    else: return list(out)

In [ ]:
assert to_int(["12",2,'99']) == [12, 2, 99]
assert to_int([[[1],[2,3]],[4,5,6]]) == [[[1], [2, 3]], [4, 5, 6]]

In [ ]:
#export

def flatten(x):
    # recursive flattens the input. Returns a list
    return list(_flatten(x))

def _flatten(x):
    for item in x:
        if isinstance(item,Iterable) and not isinstance(item, str):
            yield from flatten(item)
        else:
            yield item

In [ ]:
assert flatten([1,2,4,[99,33,[22,11]], 'f']) == [1, 2, 4, 99, 33, 22, 11, 'f']
assert flatten([[[1],[2,3]],[4,5,6]]) == [1, 2, 3, 4, 5, 6]


In [ ]:
#export
def md5(input):
    return hashlib.md5(input.encode('utf-8')).hexdigest()


In [ ]:
md5('bla')

'128ecf542a35ac5270a87dc740918404'

In [ ]:
#export
def arr_to_dict(arr):
    """
        takes in an numpy array or list and returns a dictionary with indices, values
    """
    d = {}
    for i in range(len(arr)):
        for j in range(len(arr[0])):
            d[(i,j)] = arr[i][j]
    return d


In [ ]:
#export
def reverse_dict(d):
    return {v:k for k,v in d.items()}

In [ ]:
a = {(0,0):'f'}
a.update(reverse_dict(a))
assert a == {(0, 0): 'f', 'f': (0, 0)}

In [ ]:
#export
def untar_data(path, save_dir = None):
    # will untar a file to save dir or the directory of path
    if isinstance(path,Path):
        if str(path).endswith("tgz"):
            tar = tarfile.open(fname, "r:gz")
        else:
            tar = open(path)
        tar.extractall(save_dir) if save_dir else tar.extractall(path.parent)
        tar.close()
    else:
        print('argument should be Path instance')

SyntaxError: invalid syntax (<ipython-input-13-bd3c0ba8cb98>, line 5)

In [ ]:
#export
logging.getLogger().setLevel(logging.INFO)
@contextlib.contextmanager
def timeit(description=None):
    # usage: with timeit('description') do what you like
    # will output the time taken in seconds
    tstart = time.time()
    yield
    elapsed = (time.time() - tstart)/60
    logging.info(f'{description}, time: {elapsed}')


## Data science helpers

In [ ]:
#export
def nan_inspect(df):
    # will return some sort or correlation matrix
    # this helps to quickly see where the na values are and if they are shared with multiple columns
    print(df.isnull().sum(axis=1).value_counts(sort=False))
    df_len = len(df)
    df = df.isna().copy()
    df = df.loc[:, (df != 0).any(axis=0)]
    df = df.loc[(df!=0).any(1)]
    df_nan_dict = {c : [len(df.loc[df[c] & df[c2]]) for c2 in df.columns] for c in df.columns}
    df_nan = pd.DataFrame.from_dict(df_nan_dict)
    df_nan.set_index(df.columns, inplace=True)
    df_nan.insert(0, 'total', df_nan.max(axis=0))
    df_nan.sort_values('total', inplace=True, ascending=False)
    df_nan = pd.concat([df_nan['total'] , df_nan.iloc[:,1:][list(df_nan.index)]],axis=1)
    return (df_nan/df_len).round(2)

## Grid helpers

In [ ]:
#export
def neighbors(i, diag = True,inc_self=False):
    """
     determine the neighbors, returns a set with neighboring tuples {(0,1)}
     if inc_self: returns self in results
     if diag: return diagonal moves as well
    """
    r = [1,0,-1]
    c = [1,-1,0]
    if diag:
        if inc_self: 
            return {(i[0]+dr, i[1]+dc) for dr in r for dc in c}
        else: 
            return {(i[0]+dr, i[1]+dc) for dr in r for dc in c if not (dr == 0 and dc == 0)}
    else:
        res =  {(i[0],i[1]+1), (i[0],i[1]-1),(i[0]+1,i[1]),(i[0]-1,i[1])}
        if inc_self: res.add(i)
        return res

In [ ]:
# 4 and 5 tuples
assert neighbors((0,0), inc_self=False, diag=False) == {(0, 1), (0, -1), (1, 0), (-1, 0)}
assert neighbors((0,0), inc_self=True, diag=False) == {(0, 1), (0, -1), (1, 0), (-1, 0), (0, 0)}
# # 8 and 9 tuples
assert neighbors((0,0), inc_self=False, diag=True) == {(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)}
assert neighbors((0,0), inc_self=True, diag=True) == {(1, 1), (1, -1), (1, 0), (0, 1), (0, -1), (0, 0), (-1, 1), (-1, -1), (-1, 0)}


In [ ]:
#export
def arr_neighbors(arr, diag=True, inc_self=False):
    """
    Returns a dictionary with index: set of neighbor indices
    Parameters: diag to include diagonal neighbors, inc_self to include self in result list
    Usage: for index, neighbor_indices in aoc.arr_neighbors(arr).items():
    """
    res = {}
    for i in np.ndindex(arr.shape):
        # print('hi',i, neighbors(i,diag))
        res[i] = {(x,y) for x,y in neighbors(i,diag, inc_self) if 0<=x<arr.shape[0] and 0<=y<arr.shape[1]}
    return res


In [ ]:
a = np.arange(9).reshape(3,3).astype(object)
print(a,a.shape)
n = (arr_neighbors(a))
n



[[0 1 2]
 [3 4 5]
 [6 7 8]] (3, 3)
hi (0, 0) {(0, 1), (-1, 1), (-1, 0), (-1, -1), (0, -1), (1, 0), (1, -1), (1, 1)}
hi (0, 1) {(1, 2), (-1, 1), (0, 0), (-1, 0), (-1, 2), (1, 1), (1, 0), (0, 2)}
hi (0, 2) {(1, 2), (0, 1), (-1, 1), (1, 3), (-1, 3), (-1, 2), (0, 3), (1, 1)}
hi (1, 0) {(0, 1), (0, 0), (2, -1), (2, 1), (2, 0), (0, -1), (1, -1), (1, 1)}
hi (1, 1) {(1, 2), (0, 1), (0, 0), (2, 1), (2, 0), (2, 2), (1, 0), (0, 2)}
hi (1, 2) {(0, 1), (1, 3), (0, 2), (2, 1), (2, 3), (2, 2), (0, 3), (1, 1)}
hi (2, 0) {(3, 0), (2, -1), (3, 1), (2, 1), (3, -1), (1, 0), (1, -1), (1, 1)}
hi (2, 1) {(1, 2), (3, 2), (3, 0), (3, 1), (2, 0), (2, 2), (1, 0), (1, 1)}
hi (2, 2) {(1, 2), (3, 2), (1, 3), (3, 3), (3, 1), (2, 1), (2, 3), (1, 1)}


{(0, 0): {(0, 1), (1, 0), (1, 1)},
 (0, 1): {(0, 0), (0, 2), (1, 0), (1, 1), (1, 2)},
 (0, 2): {(0, 1), (1, 1), (1, 2)},
 (1, 0): {(0, 0), (0, 1), (1, 1), (2, 0), (2, 1)},
 (1, 1): {(0, 0), (0, 1), (0, 2), (1, 0), (1, 2), (2, 0), (2, 1), (2, 2)},
 (1, 2): {(0, 1), (0, 2), (1, 1), (2, 1), (2, 2)},
 (2, 0): {(1, 0), (1, 1), (2, 1)},
 (2, 1): {(1, 0), (1, 1), (1, 2), (2, 0), (2, 2)},
 (2, 2): {(1, 1), (1, 2), (2, 1)}}

In [ ]:
#export

Dim = namedtuple('Dim',['min','max','range'])
def dimensions(obj): 
    """
     takes an iterable of iterables and returns a namedtuple with minima, maxima and range
     for example a 2d numpy array
     dim.min, dim.max and dim.range
    """
    minim = tuple(min(obj,key = lambda x:x[i])[i] for i in range(len(obj[0])))
    maxim = tuple(max(obj,key = lambda x:x[i])[i] for i in range(len(obj[0])))# max for dimensions
    ranges = tuple(maxim[i] - minim[i]  for i in range(len(obj[0])))
    res = Dim(minim,maxim,ranges)
    return res

In [ ]:
assert dimensions([[1,2,3],[10,9,8]]) == Dim(min=(1, 2, 3), max=(10, 9, 8), range=(9, 7, 5))

Example

In [ ]:
out = dimensions([[1,2,3],[10,9,8]])
out.min

(1, 2, 3)

In [ ]:
#export
def positive(*args): 
    """ 
        takes 1 or multiple lists of n coordinates and returns it normalized (getting rid of negatives)
    """
    dtype = type(args[0][0]) # support list(s) of lists and list(s) of tuples
    if len(args)==1: # only 1 argument passed
        dim = dimensions(args[0])
        obj = args[0]
        if dtype == tuple:
            return [tuple(o[i]-dim.min[i] for i in range(len(obj[0]))) for o in obj]
        if dtype == list:
            return [[o[i]-dim.min[i] for i in range(len(obj[0]))] for o in obj]
        else: print('no support for dtype',dtype)
    else: # multiple arguments passed
        dim = dimensions([i for a in args for i in a])
        if dtype == tuple:
            return ([tuple(o[i]-dim.min[i] for i in range(len(obj[0]))) for o in obj] for obj in args)

        if dtype == list: 
            return ([[o[i]-dim.min[i] for i in range(len(obj[0]))] for o in obj] for obj in args)
        else: print('no support for dtype',dtype)


positive() will only make changes along axis where negative values are detected

In [ ]:
assert positive([(0,0,0,-4),(0,0,-10,0),(0,0,0,0)]) == [(0, 0, 10, 0), (0, 0, 0, 4), (0, 0, 10, 4)]

In [ ]:
#export
def manhattan(x,y):
    return abs(x[0]-y[0])+abs(x[1]-y[1])

In [ ]:
assert manhattan((10,10),(-1,11)) == 12

In [ ]:
#export
def conv1d(arr,conv_shape,mode='same',padding=None,pad_dir='center') ->list:
    """
    Returns a list of kernel views of a string or list 
    mode == 'valid': returns only results where the kernel fits
    mode == 'same': return the same amount of items as original
    when mode =='same', default padding is the outer value
    """
    if padding:
        to_pad = padding # user specified padding
    else:
        to_pad = arr[0] # begin or end of list

    if isinstance(arr,list): # to convert a list temporarily to string
        arr_is_list = True
    else:
        arr_is_list = False

    if mode == 'valid':
        pass

    p_size = conv_shape//2
    if mode == 'same':
        if arr_is_list:
            arr = ''.join(arr)
        if isinstance(arr,str): #here the padding is applied
            if pad_dir == 'center':
                arr = to_pad*p_size+arr+to_pad*p_size
            if pad_dir == 'left':
                arr = to_pad*(conv_shape-1)+arr
            if pad_dir == 'right':
                arr = arr+to_pad*(conv_shape-1)
        else:
            return 'only string and list supported'
        if arr_is_list:
            arr = list(arr)

    if conv_shape % 2 == 1: # odd conv_shape
        return [arr[i-p_size:i+p_size+1] for i in range(p_size,len(arr)-p_size)]
    else: # even conv_shape
        return [arr[i:i+conv_shape] for i in range(0,len(arr)-conv_shape+1)]


In [ ]:
assert conv1d("12345",3,mode='valid' == ['123', '234', '345'])


In [ ]:
#export
def conv2d(arr,conv_shape,mode='valid',padding=None,pad_dir='center') ->list:
    """
    Returns a list of kernel views of a string or list 
    mode == 'valid': returns only results where the kernel fits
    mode == 'same': return the same amount of items as original
    when mode =='same', default padding is the outer value
    """
    if padding:
        to_pad = padding # user specified padding
    else:
        to_pad = arr[0] # begin or end of list

    if isinstance(arr,list) or isinstance(arr,np.ndarray): # to convert a list to numpy array
        arr_is_list = True
    else:
        arr_is_list = False

    if mode == 'valid':
        pass

    p_size = conv_shape//2
    if mode == 'same':
        if arr_is_list:
            arr = np.array(arr)
        if isinstance(arr,str): #here the padding is applied
            if pad_dir == 'center':
                arr = to_pad*p_size+arr+to_pad*p_size
            if pad_dir == 'left':
                arr = to_pad*(conv_shape-1)+arr
            if pad_dir == 'right':
                arr = arr+to_pad*(conv_shape-1)
        else:
            return 'only string and list supported'
        if arr_is_list:
            arr = list(arr)

    if conv_shape % 2 == 1: # odd conv_shape
        return [arr[i-p_size:i+p_size+1] for i in range(p_size,len(arr)-p_size)]
    else: # even conv_shape
        return [arr[i:i+conv_shape] for i in range(0,len(arr)-conv_shape+1)]


In [ ]:
conv2d(np.array([np.arange(9).reshape(3,3)]), conv_shape=2)


[]

## Specific algo's

In [ ]:
#export

def binarysearch(minim,maxim,function, flips_to_true=True): 
    """
     function needs to return a boolean whether the solution is ok
     this implementation is for function that starts with false for minim and flip to true
     for TTTTFFFF, pass set flips_to_true flag to false. This flag is important to set correct!
    """
    new = minim
    while True:
        new = (minim+maxim)//2
        print(f'to_test: {new}, min {minim}, max {maxim} ', end=' ')
        res = function(new)
        print('function returns', res)
        if not flips_to_true: res = not res
        if res:
            if new == maxim: # solution found
                if flips_to_true:
                    print('solution found',new)
                    return new
                else:
                    print('solution found',new-1)
                    return new-1
            maxim = new
        else: minim = new+1


In [ ]:
assert binarysearch(0,200, lambda x: x > 50) == 51

to_test: 100, min 0, max 200  function returns True
to_test: 50, min 0, max 100  function returns False
to_test: 75, min 51, max 100  function returns True
to_test: 63, min 51, max 75  function returns True
to_test: 57, min 51, max 63  function returns True
to_test: 54, min 51, max 57  function returns True
to_test: 52, min 51, max 54  function returns True
to_test: 51, min 51, max 52  function returns True
to_test: 51, min 51, max 51  function returns True
solution found 51


In [ ]:
assert binarysearch(0,200, lambda x: x < 50, flips_to_true=False) == 49

to_test: 100, min 0, max 200  function returns False
to_test: 50, min 0, max 100  function returns False
to_test: 25, min 0, max 50  function returns True
to_test: 38, min 26, max 50  function returns True
to_test: 44, min 39, max 50  function returns True
to_test: 47, min 45, max 50  function returns True
to_test: 49, min 48, max 50  function returns True
to_test: 50, min 50, max 50  function returns False
solution found 49


In [ ]:
#export
def deduce_matches(input_dict, option_type=str):
    """
    Takes a dict with multiple keys that have one or more options
    The trick is to start with what you know: keys with one option and remove that option for the other keys
    Continuing that process leads to every key ending up with one option (hopefully)

    Assumes: the options are strings and sort in some kind of container
    """
    found = 0
    while found < len(input_dict):
        for k, v in input_dict.items():
            if not isinstance(v, option_type) and len(v) == 1: # found one
                to_rem = v.pop()
                input_dict[k] = to_rem
                found += 1
                for _ , v2 in input_dict.items(): # remove the item from other lists
                    if not isinstance(v2, option_type) and to_rem in v2:
                        v2.remove(to_rem)
    return input_dict

## Maze algo's

In [ ]:
#export
def bfs(connections, start, goal=None, verbose=False):
    """
    Requires a connections dict with tuples with neighbors per node.
    Or a connections function returning neighbors per node

    Returns
    if goal == None:    return dict of locations with neighbor closest to start
    elif goal found:    returns path to goal
    else:               returns False
    """
    seen = set() # the locations that have been explored
    frontier = deque([start]) # the locations that still need to be visited
    # paths = {start: [start]}
    isfunction = callable(connections)
    parents = {start: None}

    def get_path(parents,start,goal):
        # print(start,goals)
        cur = goal
        path = [cur]
        while cur != start:
            cur = parents[cur]
            path.append(cur)
        path.reverse()
        return path

    while frontier:
        search = frontier.popleft()
        if isfunction: neighbors = connections(search)
        else: neighbors = connections.get(search,None)
        if neighbors:
            for n in neighbors:
                if n not in seen:
                    seen.add(n)
                    frontier.append(n)
                    # paths[n] = paths[search]+[n]
                    parents[n]= search
                    if goal and n == goal:
                        # print('goal found')
                        return get_path(parents,start,goal)
                        # return paths[goal],parents
        seen.add(search)
    if goal: return False
    else: return parents

In [ ]:
def test_bfs(input):
    if input < 0: return (0,)
    elif input > 25: return (25,)
    else:
        return (input-1, input+1, input + 20, input -20)
bfs(test_bfs, 0,goal=10) == [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

False

In [ ]:
#export
def dijkstra(connections,start, goal=None):
    """
    Requires a dict with as values a LIST of tuples (neighbor, weight)
    Or a function returning a list of tuples with neighbors and weights per node

    Returns
    if goal == None:    return all paths from start
    elif goal found:    returns path to goal
    else:               returns False
    """
    seen = set() # the locations that have been explored
    frontier = [(0,start)] # the locations that still need to be visited
    isfunction = callable(connections)
    parents = {start: (None,0)}

    def get_path(parents):
        cur = goal
        path = [cur]
        cost = parents[cur][1]
        while cur != start:
            cur = parents[cur][0]
            path.append(cur)
        path.reverse()
        return path,cost

    while frontier:
        # print('\n\n',frontier,'\n',parents)
        search_cost, search_node = heapq.heappop(frontier)
        # print('looking for', search_node,search_cost)
        if search_node == goal: break
        if isfunction: neighbors = connections(search_node)
        else: neighbors = connections.get(search_node,None)
        if neighbors:
            for n in neighbors:
                # print('n',n)
                if n[0] not in parents or n[1]+ search_cost < parents[n[0]][1]:
                    # print('updating')
                    heapq.heappush(frontier,(search_cost+n[1],n[0]))
                    # paths[n] = paths[search_node]+[n]
                    parents[n[0]]= (search_node,search_cost+n[1])
                        # return paths[goal],parents
        seen.add(search_node)
    if not goal: return parents
    elif goal in parents: return get_path(parents)
    else: return False


In [ ]:
test_dict = {1:[(2,1),(5,5)],
            2:[(1,1),(3,1)],
            3:[(2,1),(10,10)],
            5:[(1,1),(10,1)],
            10:[(3,1),(5,1)]
            }
assert dijkstra(test_dict, 1,goal=10) == ([1, 5, 10], 6)


In [ ]:
#export
def get_path(parents,start,goal):
    # print(start,goals)
    cur = goal
    path = [cur]
    while cur != start:
        cur = parents[cur]
        path.append(cur)
    path.reverse()
    return path

In [ ]:
#export
# found this on internet
def dfs(graph, start):
    visited, stack = set(), [start]
    while stack:
        vertex = stack.pop()
        if vertex not in visited:
            visited.add(vertex)
            stack.extend(graph[vertex] - visited)
    return visited

## Sequences

In [ ]:
#export
def find_pattern_in_iter(start_pattern, function, goal = None, n_iter=1000000000):
    """
        Returns when a SPECIFIED pattern has been found from a function
        If goal = None, then first time the start pattern shows up again is returned
    """
    if not goal: goal = start_pattern
    current = start_pattern
    for i in range(1,n_iter):
        current = function(current)
        # print(current)
        if current == goal:
            print('found at step',i, current)
            return i, current

In [ ]:
#export
def find_repeat(start_pattern, function, n_iter=None):
    """
        Returns when a NONSPECFIED repeating pattern has been found
    """
    if not n_iter: n_iter = round(10e20)
    seen = {start_pattern}
    current = start_pattern
    for i in range(1,n_iter):
        current = function(current)
        # print(current)
        if current in seen:
            print('found at step',i,current)
            return i,current
        seen.add(current)

In [ ]:
#export
def find_cycle(start_pattern, function):
    """
        Find cycle length of some repeating pattern, by first inspecting which item repeats when
        And subtracting the time the item was first seen
    """
    step_second, pattern = find_repeat(start_pattern, function)
    step_first, pattern = find_pattern_in_iter(pattern, Test_gen(), goal = pattern)
    return step_second - step_first

In [ ]:
#hide
class Test_gen():
    def __init__(self):
        self.results = iter([5,10,15,5,99,10])
    def __call__(self,*args):
        return next(self.results)



In [ ]:
assert find_pattern_in_iter(99,Test_gen(),n_iter=10) == (5,99)
assert find_pattern_in_iter(99,Test_gen(),goal=10, n_iter=10) == (2,10)
assert find_pattern_in_iter(99,Test_gen(),goal=5, n_iter=10) == (1,5)
assert find_repeat(99,Test_gen(),n_iter=10) == (4,5)

found at step 5 99
found at step 2 10
found at step 1 5
found at step 4 5


# Mathy functions

In [ ]:
#export
def factors(n):
    """
     return set of divisors of a number
    """

    step = 2 if n%2 else 1
    return set(reduce(list.__add__,
                ([i, n//i] for i in range(1, int(sqrt(n))+1, step) if n % i == 0)))


In [ ]:
assert factors(20) == {1, 2, 4, 5, 10, 20}


In [ ]:
def gcd(a,b):
    largest = max(a,b)
    smallest = min(a,b)
    while True:
        rest = largest % smallest
        if rest == 0:
            return prevrest
        else:
            prevrest = rest
            largest = smallest
            smallest = rest

def lcm(a):
    lcm = a[0]
    for i in a[1:]:
        lcm = lcm*i//gcd(lcm, i)
    return lcm
lcm([4,6,7])


84

In [ ]:
a = [1,2,3,8,8,8,2,3]
a.index(8)
len(a) - 1 - a[::-1].index(8)

5

In [ ]:
def power(a,b,M=None):
    # computes a**b. Actually python pow does this with optional third argument
    res = 1
    while(b):
        if b % 2 == 1: 
            res = (res * a) % M if M else res * a
            print('res',res)
        a *= a
        print('a',a)
        b //= 2
        print('b',b)
    return res
power(3,12)

a 9
b 6
a 81
b 3
res 81
a 6561
b 1
res 531441
a 43046721
b 0


531441

In [6]:
#hide
from nbdev.export import notebook2script; 
notebook2script()
!nbdev_build_lib
!nbdev_build_docs
!nbdev_clean_nbs
!git add .
!git commit -am "change future upwards"
!git push

Converted 00_core.ipynb.
Converted 01_context_free_grammar.ipynb.
Converted 02_norvig.ipynb.
Converted index.ipynb.
Converted 00_core.ipynb.
Converted 01_context_free_grammar.ipynb.
Converted 02_norvig.ipynb.
Converted index.ipynb.
converting: d:\Documenten\GitHub\adventofcode\aocutils\00_core.ipynb
converting: d:\Documenten\GitHub\adventofcode\aocutils\01_context_free_grammar.ipynb
converting: d:\Documenten\GitHub\adventofcode\aocutils\02_norvig.ipynb
converting: d:\Documenten\GitHub\adventofcode\aocutils\index.ipynb
converting d:\Documenten\GitHub\adventofcode\aocutils\index.ipynb to README.md
[main 47c0ec4] change future upwards
 3 files changed, 14 insertions(+), 7 deletions(-)
To https://github.com/jvanelteren/aocutils.git
   68e7b8a..47c0ec4  main -> main
